In [1]:
#| include: false

import duckdb
import pandas as pd
%load_ext sql

In [2]:
#| include: false

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [3]:
#| include: false

%sql duckdb:///:memory:

%sql CREATE TABLE user_actions AS SELECT * FROM '00_data/sql/user_actions.csv';
%sql CREATE TABLE courier_actions AS SELECT * FROM '00_data/sql/courier_actions.csv';
%sql CREATE TABLE orders AS SELECT * FROM '00_data/sql/orders.csv';
%sql CREATE TABLE users AS SELECT * FROM '00_data/sql/users.csv';
%sql CREATE TABLE couriers AS SELECT * FROM '00_data/sql/couriers.csv';
%sql CREATE TABLE products AS SELECT * FROM '00_data/sql/products.csv';

,Count
0,87


# SQL {#sec-sql}

SQL (анг. *Structured query language*) - мова запитів на якому ми спілкуємось з базами даних.

База даних - це сховище даних, де зберігаються наші "таблиці".

## Схема бази даних

У цьому розділі ми будемо використовувати набори даних, які представляють сервіс з доставки продуктів. Тут зберігається документація, котра допоможе розібратися з ними.

Для запуску бази даних, я використовую [DuckDB](https://duckdb.org/). Самі файли з даними можна знайти в [репозиторії](https://github.com/Aranaur/py4ds/tree/main/00_data/sql).
Для локального запуску бази даних необхідно виконати наступний код:

In [4]:
#| eval: false
#| code-fold: true

# Встановлення та імпорт необхідних пакетів
!pip install --quiet duckdb
!pip install --quiet jupysql 
!pip install --quiet duckdb-engine
!pip install --quiet pandas
!pip install --quiet matplotlib

import duckdb
import pandas as pd

# Імпорт Jupyter-розширення jupysql для створення SQL комірок
%load_ext sql

# Налаштування jupysql, щоб дані поверталися у вигляді DataFrame Pandas з меншим виводом
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

# Підключення до DuckDB в режимі "in-memory"
%sql duckdb:///:memory:


# Завантаження даних з файлів
%sql CREATE OR REPLACE TABLE user_actions AS SELECT * FROM read_csv('00_data/sql/user_actions.csv', header=True, columns={'user_id': 'INT', 'order_id': 'INT', 'action': 'VARCHAR', 'time': 'TIMESTAMP'}, timestampformat='%d/%m/%y %H:%M');
%sql CREATE OR REPLACE TABLE courier_actions AS SELECT * FROM read_csv('00_data/sql/user_actions.csv', header=True, columns={'courier_id': 'INT', 'order_id': 'INT', 'action': 'VARCHAR', 'time': 'TIMESTAMP'}, timestampformat='%d/%m/%y %H:%M');
%sql CREATE OR REPLACE TABLE orders AS SELECT * FROM read_csv('00_data/sql/orders.csv', header=True, columns={'order_id': 'INT', 'creation_time': 'TIMESTAMP', 'product_ids': 'INT[]'}, timestampformat='%d/%m/%y %H:%M');
%sql CREATE OR REPLACE TABLE users AS SELECT * FROM read_csv('00_data/sql/users.csv', header=True, columns={'user_id': 'INT', 'birth_date': 'DATE', 'sex': 'VARCHAR'}, dateformat='%d/%m/%y');
%sql CREATE OR REPLACE TABLE couriers AS SELECT * FROM read_csv('00_data/sql/couriers.csv', header=True, columns={'courier_id': 'INT', 'birth_date': 'DATE', 'sex': 'VARCHAR'}, dateformat='%d/%m/%y');
%sql CREATE OR REPLACE TABLE products AS SELECT * FROM read_csv('00_data/sql/products.csv', header=True, columns={'product_id': 'INT', 'name': 'VARCHAR', 'price': 'DOUBLE'});

На схемі продемонстровані зв'язки між таблицями, а також опис даних:

```{mermaid}
erDiagram
    orders }|..|{ products : product_ids-product_id
    orders }|..|{ courier_actions : order_id
    users }|..|{ user_actions : user_id
    user_actions }|..|{ orders : order_id
    user_actions }|..|{ courier_actions : time
    courier_actions }|..|{ courier : product
    
    users {
        DATE birth_date
        VARCHAR sex
        INT user_id
    }
    user_actions {
        INT user_id
        VARCHAR actions
        INT order_id
        TIMESTAMP time
    }
    orders {
        INT order_id
        ARRAY product_ids
        TIMESTAMP creation_time
    }
    products {
        INT product_id
        NUMERIC price
        VARCHAR name
    }
    courier_actions {
        INT courier_id
        VARCHAR action
        INT order_id
        TIMESTAMP time
    }
    courier {
        INT courier_id
        VARCHAR sex
        DATE birth_date
    }
```

## Типи даних
В таблицях можуть зберігатися різні типи даних: цілі і дробові числа, текст, дати, масиви чисел. У цих даних ви зустрінетесь з наступними типами:

In [5]:
# | label: sql-df
# | echo: false

import pandas as pd

sql_data_types = pd.DataFrame(
    {
        "Типи даних": [
            "INT",
            "NUMERIC / DECIMAL",
            "VARCHAR",
            "DATE",
            "TIMESTAMP",
            "[]",
        ],
        "Опис": [
            "Ціле число",
            "Дійсне число",
            "Текст",
            "Дата з точністю до дня",
            "Дата з точністю до секунди",
            "Масив",
        ],
        "Приклад": [
            "id користувача: 123",
            "Вартість товару: 120.55 ",
            "Дія із замовленням: «create_order»",
            "Дата народження користувача: 25/03/91",
            "Час реєстрації у додатку: 24/08/22 01:52:24",
            "Список id товаров у замовленні: [1, 13, 22]",
        ],
    }
)

user_actions = pd.DataFrame(
    {
        "Стовпчик": ["user_id", "order_id", "action", "time"],
        "Тип даних": ["INT", "INT", "VARCHAR(50)", "TIMESTAMP"],
        "Опис": [
            "id користувача",
            "id замовлення",
            'дія користувача із замовленням; "create_order" - створення замовлення, "cancel_order" - скасування замовлення',
            "час дії",
        ],
    }
)

courier_actions = pd.DataFrame(
    {
        "Стовпчик": ["courier_id", "order_id", "action", "time"],
        "Тип даних": ["INT", "INT", "VARCHAR(50)", "TIMESTAMP"],
        "Опис": [
            "id кур'єра",
            "id замовлення",
            "дія кур'єра із замовленням; 'accept order' - прийняття замовлення, 'deliver order' - доставка замовлення",
            "час дії",
        ],
    }
)

orders = pd.DataFrame(
    {
        "Стовпчик": ["order_id", "creation_time", "product_ids"],
        "Тип даних": ["INT", "TIMESTAMP", "integer[]"],
        "Опис": [
            "id замовлення",
            "час створення замовлення",
            "список id товарів у замовленні",
        ],
    }
)

users = pd.DataFrame(
    {
        "Стовпчик": ["user_id", "birth_date", "sex"],
        "Тип даних": ["INT", "DATE", "VARCHAR(50)"],
        "Опис": ["id користувача", "дата народження", "стать"],
    }
)

couriers = pd.DataFrame(
    {
        "Стовпчик": ["courier_id", "birth_date", "sex"],
        "Тип даних": ["INT", "DATE", "VARCHAR(50)"],
        "Опис": ["id кур'єра", "дата народження", "стать"],
    }
)

products = pd.DataFrame(
    {
        "Стовпчик": ["product_id", "name", "price"],
        "Тип даних": ["INT", "VARCHAR(50)", "	FLOAT(4)"],
        "Опис": ["id продукту", "назва товару", "ціна товару"],
    }
)

In [6]:
# | label: sql-data-types
# | echo: false
sql_data_types

,Типи даних,Опис,Приклад
0,INT,Ціле число,id користувача: 123
1,NUMERIC / DECIMAL,Дійсне число,Вартість товару: 120.55
2,VARCHAR,Текст,Дія із замовленням: «create_order»
3,DATE,Дата з точністю до дня,Дата народження користувача: 25/03/91
4,TIMESTAMP,Дата з точністю до секунди,Час реєстрації у додатку: 24/08/22 01:52:24
5,[],Масив,"Список id товаров у замовленні: [1, 13, 22]"


Більш детально почитати про типи даних можна за [посиланням](https://www.postgresqltutorial.com/postgresql-tutorial/postgresql-data-types/)

## Структура та наповнення таблиць

`user_actions` – дії користувачів із замовленнями.

In [7]:
# | label: sql-user-actions
# | echo: false
user_actions

,Стовпчик,Тип даних,Опис
0,user_id,INT,id користувача
1,order_id,INT,id замовлення
2,action,VARCHAR(50),"дія користувача із замовленням; ""create_order""..."
3,time,TIMESTAMP,час дії


`courier_actions` – дії кур'єрів із замовленнями.

In [8]:
# | label: sql-courier-actions
# | echo: false
courier_actions

,Стовпчик,Тип даних,Опис
0,courier_id,INT,id кур'єра
1,order_id,INT,id замовлення
2,action,VARCHAR(50),дія кур'єра із замовленням; 'accept order' - п...
3,time,TIMESTAMP,час дії


`orders` - інформація про замовлення.

In [9]:
# | label: sql-orders
# | echo: false
orders

,Стовпчик,Тип даних,Опис
0,order_id,INT,id замовлення
1,creation_time,TIMESTAMP,час створення замовлення
2,product_ids,integer[],список id товарів у замовленні


`users` - інформація про користувачів.

In [10]:
# | label: sql-users
# | echo: false
users

,Стовпчик,Тип даних,Опис
0,user_id,INT,id користувача
1,birth_date,DATE,дата народження
2,sex,VARCHAR(50),стать


`couriers` - інформація про кур'єрів.

In [11]:
# | label: sql-couriers
# | echo: false
couriers

,Стовпчик,Тип даних,Опис
0,courier_id,INT,id кур'єра
1,birth_date,DATE,дата народження
2,sex,VARCHAR(50),стать


`products` - інформація про товари, які доставляє сервіс.

In [12]:
# | label: sql-products
# | echo: false
couriers

,Стовпчик,Тип даних,Опис
0,courier_id,INT,id кур'єра
1,birth_date,DATE,дата народження
2,sex,VARCHAR(50),стать


::: {.callout-note}
У дужках типу `VARCHAR` вказано максимально допустиму кількість символів у тексті. У типу даних `NUMERIC` у дужках вказано загальну кількість символів.
:::